In [54]:
import numpy as np
from sgp4.api import Satrec 
from astropy.time import Time
import astropy.units as u 

from poliastro.bodies import Earth
from poliastro.twobody import Orbit

from astropy.coordinates import get_sun
from poliastro.constants import R_earth
import astropy.units as u

In [56]:
line_1 = "1 63223U 25052P   25244.59601767  .00010814  00000-0  51235-3 0  9991"
line_2 = "2 63223  97.4217 137.0451 0006365  74.2830 285.9107 15.19475170 25990"

In [58]:
sat = Satrec.twoline2rv(line_1, line_2)

In [60]:
start_time = Time("2025-09-01 00:00:00", scale="utc") #Tracker's epoch 

time_step = 60 
total_steps = int(((24*3600)/time_step) + 1) 

time_step_array = start_time + np.arange(total_steps) * time_step * u.s

In [62]:
pos_vec_list = np.empty((total_steps, 3))
vel_vec_list = np.empty((total_steps, 3))
errors = np.zeros(total_steps, dtype=int)

for i, t in enumerate(time_step_array):
    e, r, v = sat.sgp4(t.jd1, t.jd2)
    errors[i] = e
    if e == 0:
        pos_vec_list[i] = r
        vel_vec_list[i] = v
    else:
        pos_vec_list[i] = np.nan
        vel_vec_list[i] = np.nan
        print(f"SGP4 error {e} at step {i}, time {t.iso}")

In [64]:
sptracker_epoch = Time("2025-09-01 00:00:00", scale="utc")

sptracker_orbit = Orbit.from_classical(
    Earth,
    6878 * u.km,          #SMA
    0 * u.one,            #Eccentricity
    97.4 * u.deg,         #Inclination
    72.628 * u.deg,       #RAAN
    331.7425 * u.deg,     #Arg of Perigee
    0 * u.deg,            #Mean Anomaly 
    epoch= sptracker_epoch
)

In [66]:
print(sptracker_orbit.r)
print(sptracker_orbit.v)


[ 1408.60036661  5907.21800977 -3229.16465857] km
[1.90033693 3.18185482 6.64962232] km / s


In [68]:
sptracker_pos = np.empty((total_steps, 3))
sptracker_vel = np.empty((total_steps, 3))

for i, t in enumerate(time_step_array):
    propagated = sptracker_orbit.propagate(t - sptracker_orbit.epoch)
    sptracker_pos[i] = propagated.r.to(u.km).value
    sptracker_vel[i] = propagated.v.to(u.km / u.s).value

In [69]:
rel_vec = pos_vec_list - sptracker_pos
rel_dist = np.linalg.norm(rel_vec, axis=1)

valid = np.isfinite(rel_dist)

In [74]:
eps = 1e-9

vel_mag = np.linalg.norm(sptracker_vel, axis=1)
vel_mag_copy = vel_mag.copy()
vel_mag_copy[vel_mag_copy < eps] = np.nan
vel_unit = sptracker_vel  / vel_mag_copy[:, None]

rel_mag_copy = rel_dist.copy()
rel_mag_copy[rel_mag_copy < eps] = np.nan
rel_unit = rel_vec / rel_mag_copy[:, None]  

In [76]:
#Dot product 
cos_theta = np.sum(vel_unit * rel_unit, axis=1)

#Range
cos_theta = np.clip(cos_theta, -1.0, 1.0)

theta_deg = np.degrees(np.arccos(cos_theta))

theta_deg[~valid] = np.nan

In [78]:
half_fov = 15.0

in_fov = (theta_deg <= half_fov) & np.isfinite(theta_deg)

In [80]:
#Entry and exit edges 
b = in_fov.astype(int)
difference = np.diff(b, prepend=0)

enter_index = np.where(difference == 1)[0] #Entry index (false to true) 
exit_index  = np.where(difference == -1)[0] #Exit index (true to false)

#In case the object is within FOV before the simulation
if in_fov[0]:
    enter_index = np.insert(enter_index, 0, 0)

#In case the object stays within the FOV at the end of the simulation
if in_fov[-1]:
    exit_index = np.append(exit_index, len(in_fov)-1)

crossings = []
for start_i, end_i in zip(enter_index, exit_index):
    start_time = time_step_array[start_i]
    end_time   = time_step_array[end_i]
    duration_s = (end_time - start_time).to('s').value

    crossings.append({
        "start_index": int(start_i),
        "end_index": int(end_i),
        "start_time": start_time.iso,
        "end_time": end_time.iso,
        "duration_sec": float(duration_s)
    })


if crossings:
    print(f"Found {len(crossings)} crossing(s):")
    for k, ev in enumerate(crossings, 1):
        print(f"{k}. {ev['start_time']} -> {ev['end_time']}, duration {ev['duration_sec']:.0f}s, min angle {ev['min_angle_deg']:.2f}°")
else:
    print("No crossings found in the time window.")

No crossings found in the time window.


In [82]:
#Range check
range_check = rel_dist <= 1000

In [84]:
#Sunlight check

sun_vec = np.empty_like(pos_vec_list) 

for i, t in enumerate(time_step_array):
    sun_icrf = get_sun(t)             
    sun_vec[i] = sun_icrf.cartesian.xyz.to(u.km).value

In [86]:
# Vector from Earth to Sun and to object
r_obj = pos_vec_list
r_sun = sun_vec

# Compute dot product to check alignment
r_sun_unit = r_sun / np.linalg.norm(r_sun, axis=1)[:, None]
proj = np.sum(r_obj * r_sun_unit, axis=1)  # projection of object vector along sun direction
shadow = (proj > 0) & (np.linalg.norm(r_obj - r_sun_unit * proj[:, None], axis=1) < R_earth.value)
sunlit = ~shadow


In [88]:
visible = in_fov & range_check & sunlit


In [90]:
b = visible.astype(int)
diff = np.diff(b, prepend=0)

enter_vis = np.where(diff == 1)[0]
exit_vis  = np.where(diff == -1)[0]

if visible[0]:
    enter_vis = np.insert(enter_vis, 0, 0)
if visible[-1]:
    exit_vis = np.append(exit_vis, len(visible)-1)

visible_crossings = []
for s, e in zip(enter_vis, exit_vis):
    start_t = time_step_array[s]
    end_t   = time_step_array[e]
    duration = (end_t - start_t).to('s').value
    visible_crossings.append({
        "start_time": start_t.iso,
        "end_time": end_t.iso,
        "duration_s": float(duration)
    })

if visible_crossings:
    print(f"Found {len(visible_crossings)} visible crossings:")
    for k, ev in enumerate(visible_crossings, 1):
        print(f"{k}. {ev['start_time']} -> {ev['end_time']} ({ev['duration_s']:.0f}s)")
else:
    print("No visible crossings found in this 24-hour period.")


No visible crossings found in this 24-hour period.
